In [3]:
import pandas as pd
df = pd.read_csv('../vessel_movements_dataset.csv')
df.head()

,vessel_id,vessel_type_new,timestamp,timestamp_epoch,latitude,longitude,speed_knots,in_anchorage,in_port_boundary,safety_score,...,aux_engine_fuel_type,boil_engine_fuel_type,engine_type,mep,vref,sfc_me,sfc_ae,sfc_ab,ael,abl
0,10498900,Chemical/Products Tanker,2025/03/29 19:02:27+00,1743274947,1.331667,104.660000,12.59,NaN,Singapore,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
1,10498900,Chemical/Products Tanker,2025/03/29 20:02:27+00,1743278547,1.280000,104.861667,12.47,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
2,10498900,Chemical/Products Tanker,2025/03/29 21:02:28+00,1743282148,1.155000,105.026667,12.55,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
3,10498900,Chemical/Products Tanker,2025/03/29 22:02:25+00,1743285745,1.015000,105.180000,12.47,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
4,10498900,Chemical/Products Tanker,2025/03/29 23:02:27+00,1743289347,0.865000,105.323333,12.52,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125


In [11]:
df.columns

Index(['vessel_id', 'vessel_type_new', 'timestamp', 'timestamp_epoch',
       'latitude', 'longitude', 'speed_knots', 'in_anchorage',
       'in_port_boundary', 'safety_score', 'dwt', 'fuel_category',
       'main_engine_fuel_type', 'aux_engine_fuel_type',
       'boil_engine_fuel_type', 'engine_type', 'mep', 'vref', 'sfc_me',
       'sfc_ae', 'sfc_ab', 'ael', 'abl '],
      dtype='object')

In [38]:
print(df['main_engine_fuel_type'].unique())
print(df['aux_engine_fuel_type'].unique())
print(df['boil_engine_fuel_type'].unique())

['Methanol' 'DISTILLATE FUEL' 'LNG' 'Ammonia' 'Ethanol' 'LPG (Butane)'
 'Hydrogen' 'LPG (Propane)']
['DISTILLATE FUEL']
['DISTILLATE FUEL']


In [32]:
# df = your vessel movement dataset

# Get unique vessel IDs
df_latest = (
    df.sort_values('timestamp')
      .groupby('vessel_id', as_index=False)
      .tail(1)
)

df_first = (
    df.sort_values('timestamp')
      .groupby('vessel_id', as_index=False)
      .head(1)
)

df_vessel = pd.concat([df_first, df_latest])
id_map = {id_val: i for i, id_val in enumerate(df['vessel_id'].unique())}

# Create a temporary rank column based on the map
df_vessel['id_rank'] = df_vessel['vessel_id'].map(id_map)

# Sort by the rank, then by the second field (e.g., 'Date' or 'Value')
df_vessel = df_vessel.sort_values(by=['id_rank', 'timestamp']).drop(columns='id_rank')

# df_latest.to_csv('../lastVesselInstance.csv', index=False)
df_vessel.to_csv('../vessels_info.csv', index=False)

### Step 1

In [20]:
# step 1
step1_df = df.copy()

def classify_movement(row):
    in_anchorage = (
        0 if pd.isna(row['in_anchorage']) else 1
    )

    in_port_boundary = (
        0 if pd.isna(row['in_port_boundary']) else 1
    )

    speed = row['speed_knots']

    if in_anchorage == 1 and speed < 1:
        return 'Anchorage'
    elif in_port_boundary == 1 and speed > 1:
        return 'Maneuver'
    elif in_port_boundary == 0 and speed >= 1:
        return 'Transit'
    else:
        return 'Drifting'
step1_df['movement_mode'] = step1_df.apply(classify_movement, axis=1)
step1_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)

## Step 2

In [21]:
step2_df = step1_df.copy()
step2_df['timestamp'] = pd.to_datetime(step2_df['timestamp'], format='mixed')
# step2_df = step2_df.sort_values(['vessel_id', 'timestamp'])

step2_df['activity_hours'] = (
    step2_df.groupby('vessel_id')['timestamp']
      .diff()
      .dt.total_seconds()
      .div(3600)
      .fillna(0)
)
step2_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)


## Step 4

In [70]:
import pandas as pd
# 4a
cal_fac_df = pd.read_excel('../calculation_factors.xlsx', sheet_name='Cf')
cal_fac_df


step4_df = pd.read_csv('../vessel_movements_dataset_modified.csv')
cal_fac_df.dropna()
cal_fac_df.at[1, 'Fuel Type'] = 'DISTILLATE FUEL'
step4_df = step4_df.rename(columns={'abl ': 'abl'})


# ['Methanol' 'DISTILLATE FUEL' 'LNG' 'Ammonia' 'Ethanol' 'LPG (Butane)'
#  'Hydrogen' 'LPG (Propane)']
# ['DISTILLATE FUEL']
# ['DISTILLATE FUEL']
lcv_map = (
    cal_fac_df
    .set_index('Fuel Type')['LCV (MJ/kg)']
    .to_dict()
)

lcv_map

step4_df['lcv_me'] = step4_df['main_engine_fuel_type'].map(lcv_map)
step4_df['lcv_ae'] = step4_df['aux_engine_fuel_type'].map(lcv_map)
step4_df['lcv_ab'] = step4_df['boil_engine_fuel_type'].map(lcv_map)

DISTILLATE_LCV = 42.7

step4_df['sfc_adjusted_me'] = step4_df['sfc_me'] * (DISTILLATE_LCV / step4_df['lcv_me'])
step4_df['sfc_adjusted_ae'] = step4_df['sfc_ae'] * (DISTILLATE_LCV / step4_df['lcv_ae'])
step4_df['sfc_adjusted_ab'] = step4_df['sfc_ab'] * (DISTILLATE_LCV / step4_df['lcv_ab'])

step4_df = step4_df.drop(['lcv_me', 'lcv_ae', 'lcv_ab'], axis=1)

# 4b fuel concumption

step4_df['me_fual_consumption(tonnes)'] = (step4_df['load_factor'] * step4_df['mep'] * step4_df['sfc_adjusted_me'] * step4_df['activity_hours']) / 1000000
step4_df['ae_fual_consumption(tonnes)'] = (step4_df['ael'] * step4_df['sfc_adjusted_ae'] * step4_df['activity_hours']) / 1000000
step4_df['ab_fual_consumption(tonnes)'] = (step4_df['abl'] * step4_df['sfc_adjusted_ab'] * step4_df['activity_hours']) / 1000000

step4_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)



In [31]:
step6_df['main_engine_fuel_type'].unique()

array(['Methanol', 'DISTILLATE FUEL', 'LNG', 'Ammonia', 'Ethanol',
       'LPG (Butane)', 'Hydrogen', 'LPG (Propane)'], dtype=object)

## Step 6 

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
step6_df.columns

Index(['vessel_id', 'vessel_type_new', 'timestamp', 'timestamp_epoch',
       'latitude', 'longitude', 'speed_knots', 'in_anchorage',
       'in_port_boundary', 'safety_score', 'dwt', 'fuel_category',
       'main_engine_fuel_type', 'aux_engine_fuel_type',
       'boil_engine_fuel_type', 'engine_type', 'mep', 'vref', 'sfc_me',
       'sfc_ae', 'sfc_ab', 'ael', 'abl', 'movement_mode', 'activity_hours',
       'max_speed', 'load_factor', 'sfc_adjusted_me', 'sfc_adjusted_ae',
       'sfc_adjusted_ab', 'me_fual_consumption(tonnes)',
       'ae_fual_consumption(tonnes)', 'ab_fual_consumption(tonnes)',
       'percent_lf', 'llaf_CO2', 'llaf_CH4', 'llaf_N2O', 'total_co2',
       'total_co2_eq'],
      dtype='object')

In [68]:
fuel_cost = pd.read_excel('../calculation_factors.xlsx', sheet_name='Fuel cost')
step6_df = pd.read_csv('../vessel_movements_dataset_modified.csv')

fuel_cost.at[1, 'Fuel Type'] = 'DISTILLATE FUEL'
cost_map = (
    fuel_cost
    .set_index('Fuel Type')['Cost per GJ (USD)']
    .to_dict()
)
lcv_map = (
    fuel_cost
    .set_index('Fuel Type')['LCV (MJ/kg)']
    .to_dict()
)

# 6a
step6_df['cost_per_GJ'] = step6_df['main_engine_fuel_type'].map(cost_map)
step6_df['lcv'] = step6_df['main_engine_fuel_type'].map(lcv_map)

step6_df['cost_per_t_fuel'] = step6_df['cost_per_GJ'] * step6_df['lcv']
step6_df['total_fuel_cost_USD'] = (step6_df['me_fual_consumption(tonnes)']+step6_df['ae_fual_consumption(tonnes)']+step6_df['ab_fual_consumption(tonnes)'])*step6_df['cost_per_t_fuel']

# 6b
# Total Carbon cost (USD) = Total Equivalent CO2 emissions * Carbon cost per ton (2024)
step6_df['total_carbon_cost_USD'] = step6_df['total_co2_eq']*80

# 6c
# a.	Asset depreciation rate (r) = 8% per annum
# b.	Life of each ship (N) = 30 years
# c.	Cost of each ship = Distillate fuel ship cost as per DWT * Multiplier factor (M) as per main_engine_fuel_type
# d.	Salvage Cost (S) i.e. End of life value = 10% of cost of ship
# e.	Capital Recovery Factor (crf) = {r*(1+r)^N}/{((1+r)^N)-1}
# f.	Amortized annual ownership cost of ship (million USD) = ((P-S)*CRF)+(r*S)

def determine_soc(row):
    dwt = row['dwt']
    fuel_type = row['main_engine_fuel_type']
    cost = 90
    if dwt<40000:
        cost =  35
    elif dwt<55000:
        cost =  53
    elif dwt<80000:
        cost =  80
    elif dwt<12000:
        cost =  78
    
    multiplier = 1.0

    if fuel_type == 'Methanol':
        multiplier = 1.3
    elif fuel_type == 'LNG':
        multiplier = 1.4
    elif fuel_type == 'Ammonia':
        multiplier = 1.4
    elif fuel_type == 'Ethanol':
        multiplier = 1.2
    elif fuel_type == 'LPG (Butane)':
        multiplier = 1.35
    elif fuel_type == 'Hydrogen':
        multiplier = 1.1
    elif fuel_type == 'LPG (Propane)':
        multiplier = 1.3
    
    return cost*multiplier

    
step6_df['ship_ownership_cost_USD_M'] = step6_df.apply(determine_soc, result_type='reduce',axis=1)
r = 0.08
n = 30
s = 0.01
crf = (r*(1+r)**n) / (((1+r)**n)-1.0)
# cost = ((P-s*P)*crf)+(r*s*P)
step6_df['amortised_SOC_USD_monthly'] = (((step6_df['ship_ownership_cost_USD_M']-s*step6_df['ship_ownership_cost_USD_M'])*crf)+(r*(s*step6_df['ship_ownership_cost_USD_M'])))/12 * 1000000

# step6_df[['dwt', 'main_engine_fuel_type']]

vessel_totals = (
    step6_df.groupby('vessel_id', as_index=False)
      .agg(
          total_fuel_cost_USD=('total_fuel_cost_USD', 'sum'),
          total_carbon_cost_USD=('total_carbon_cost_USD', 'sum'),
          amortised_SOC_USD_monthly=('amortised_SOC_USD_monthly', 'first')

      )
)

vessel_totals['monthly_cost_USD'] = vessel_totals['total_fuel_cost_USD'] + vessel_totals['total_carbon_cost_USD'] + vessel_totals['amortised_SOC_USD_monthly']
step6_df = step6_df.merge(
    vessel_totals[['vessel_id', 'monthly_cost_USD']],
    on='vessel_id',
    how='left'
)

# 6e
safe = pd.read_excel('../calculation_factors.xlsx', sheet_name='Safety score adjustment')
safe = {
    1: 0.1,
    2: 0.05,
    3: 0,
    4: -0.02,
    5: -0.05,
}
step6_df['risk_adjustment_rate'] = step6_df['safety_score'].map(safe)
step6_df['risk_premium_USD'] = step6_df['monthly_cost_USD'] * step6_df['risk_adjustment_rate']

# 6d
step6_df['adjusted_monthly_cost'] = step6_df['monthly_cost_USD'] + step6_df['risk_premium_USD']

step6_df.to_csv('../vessel_movements_dataset_modified.csv')